# Costruisci un algoritmo di raccomandazione from scratch

## Parsing del dataset Netflix

In questa fase, importiamo e interpretiamo il file CSV contenente i titoli di Netflix, strutturando ogni riga come un dizionario Python.

L'uso di [csv.DictReader](https://docs.python.org/3/library/csv.html#csv.DictReader) consente di leggere direttamente i valori associandoli alle intestazioni delle colonne, facilitando così l'accesso ai campi tramite nomi simbolici.

Durante il parsing:

- I campi testuali come `cast` e `listed_in` vengono convertiti in liste.
- I valori mancanti vengono gestiti e ripuliti con `.strip()` e controlli condizionali.
- Il campo `release_year` viene convertito in intero solo se rappresenta un numero valido.

Questa standardizzazione iniziale è essenziale per rendere i dati utilizzabili nei successivi step di analisi e raccomandazione.


In [1]:
# Step 0

import csv
import json 
def carica_dati_netflix(path):
    dataset = []

    with open(path, 'r', encoding='utf-8') as f:
        reader = csv.DictReader(f)

        for row in reader:
            record = {
                'show_id': row.get('show_id', '').strip(),
                'type': row.get('type', '').strip(),
                'title': row.get('title', '').strip(),
                'director': row.get('director', '').strip(),
                'cast': [c.strip() for c in row.get('cast', '').split(',')] if row.get('cast') else [],
                'country': row.get('country', '').strip(),
                'date_added': row.get('date_added', '').strip(),
                'release_year': int(row['release_year']) if row.get('release_year', '').isdigit() else None,
                'rating': row.get('rating', '').strip(),
                'duration': row.get('duration', '').strip(),
                'listed_in': [g.strip() for g in row.get('listed_in', '').split(',')] if row.get('listed_in') else [],
                'description': row.get('description', '').strip()
            }

            dataset.append(record)

    return dataset

# Esempio d'uso
dati = carica_dati_netflix('data/netflix_titles.csv')
print(json.dumps(dati[7659], indent=2))


{
  "show_id": "s7660",
  "type": "Movie",
  "title": "Once Upon a Time in the West",
  "director": "Sergio Leone",
  "cast": [
    "Henry Fonda",
    "Charles Bronson",
    "Claudia Cardinale",
    "Jason Robards",
    "Gabriele Ferzetti",
    "Paolo Stoppa",
    "Woody Strode",
    "Jack Elam",
    "Keenan Wynn",
    "Frank Wolff",
    "Lionel Stander"
  ],
  "country": "Italy, United States",
  "date_added": "November 20, 2019",
  "release_year": 1968,
  "rating": "PG-13",
  "duration": "166 min",
  "listed_in": [
    "Action & Adventure",
    "Classic Movies",
    "International Movies"
  ],
  "description": "In this epic spaghetti Western, a flinty gunslinger is hired by a railroad tycoon to kill anyone standing in the way of his trans-American iron horse."
}


## Raccomandazioni per genere (filtraggio semplice)

Queste due funzioni implementano una forma elementare di **sistema di raccomandazione basato su contenuto**, sfruttando il campo `listed_in` del dataset, che rappresenta i generi associati a ciascun titolo.

### `raccomanda_per_genere`

Filtra i titoli per singolo genere (es. "Drama"). Il confronto è case-insensitive e si basa su un'equivalenza diretta tra il genere richiesto e quelli associati al titolo. I risultati sono ordinati per anno di uscita in ordine decrescente.

### `raccomanda_per_generi`

Supporta la richiesta di più generi contemporaneamente (es. "Drama, Thriller"). L'algoritmo include tutti i titoli che contengono **almeno uno** dei generi richiesti.

Queste funzioni non considerano ancora alcuna misura di similarità, distanza o vettorizzazione. Sono esempi di **filtraggio deterministico**, utile come baseline per confrontare approcci successivi più sofisticati.


In [2]:
# Step 1

def raccomanda_per_genere(dati, genere_input):
    genere_input = genere_input.lower()
    risultati = []

    for record in dati:
        generi = [g.lower() for g in record['listed_in']]
        if genere_input in generi:
            risultati.append(record)

    risultati_ordinati = sorted(risultati, key=lambda r: r['release_year'] or 0, reverse=True)

    for r in risultati_ordinati:
        print(f"{r['title']} ({r['release_year']}) — {', '.join(r['listed_in'])}")

def raccomanda_per_generi(dati, input_generi):
    # Pulisci e dividi i generi richiesti
    generi_richiesti = [g.strip().lower() for g in input_generi.split(',')]

    risultati = []

    for record in dati:
        generi_film = [g.lower() for g in record['listed_in']]
        
        # Verifica se almeno un genere combacia
        if any(gen in generi_film for gen in generi_richiesti):
            risultati.append(record)

    # Ordina per anno decrescente
    risultati_ordinati = sorted(risultati, key=lambda r: r['release_year'] or 0, reverse=True)

    # Stampa risultati
    for r in risultati_ordinati:
        print(f"{r['title']} ({r['release_year']}) — {', '.join(r['listed_in'])}")

# ESEMPIO USO
generi = input("Inserisci uno o più generi separati da virgola (es. Drama, Thriller): ")
raccomanda_per_generi(dati, generi)


# ESEMPIO D'USO
genere = input("Inserisci un genere (es. Dramas, Thrillers): ")
raccomanda_per_genere(dati, genere)


Inserisci uno o più generi separati da virgola (es. Drama, Thriller):  dramas


The Starling (2021) — Comedies, Dramas
Je Suis Karl (2021) — Dramas, International Movies
Ankahi Kahaniya (2021) — Dramas, Independent Movies, International Movies
The Father Who Moves Mountains (2021) — Dramas, International Movies, Thrillers
The Stronghold (2021) — Action & Adventure, Dramas, International Movies
Tughlaq Durbar (Telugu) (2021) — Comedies, Dramas, International Movies
JJ+E (2021) — Dramas, International Movies, Romantic Movies
Worth (2021) — Dramas
Thimmarusu (2021) — Dramas, International Movies
The Water Man (2021) — Children & Family Movies, Dramas
Man in Love (2021) — Dramas, International Movies, Romantic Movies
Sweet Girl (2021) — Action & Adventure, Dramas
I missed you: Director's Cut (2021) — Dramas, Independent Movies, International Movies
Mimi (2021) — Comedies, Dramas, International Movies
African America (2021) — Dramas, Independent Movies, International Movies
The Last Letter From Your Lover (2021) — Dramas, Romantic Movies
Cousins (2021) — Dramas
The Tam

Inserisci un genere (es. Dramas, Thrillers):  drama


In [5]:
dati[:1]

[{'show_id': 's1',
  'type': 'Movie',
  'title': 'Dick Johnson Is Dead',
  'director': 'Kirsten Johnson',
  'cast': [],
  'country': 'United States',
  'date_added': 'September 25, 2021',
  'release_year': 2020,
  'rating': 'PG-13',
  'duration': '90 min',
  'listed_in': ['Documentaries'],
  'description': 'As her father nears the end of his life, filmmaker Kirsten Johnson stages his death in inventive and comical ways to help them both face the inevitable.'}]

## Raccomandazione basata su similarità nel feature space

A questo punto, passiamo da un filtraggio simbolico a una rappresentazione quantitativa. Ogni film viene proiettato in uno **spazio delle feature** definito dai generi presenti nel dataset.

### One Hot Encoding binario

Si costruisce un **vocabolario** di tutti i generi presenti (`costruisci_vocabolario_generi`) e ogni film viene rappresentato da un vettore binario (`vettore_generi`) della stessa lunghezza del vocabolario:

- 1 se il film appartiene al genere in quella posizione
- 0 altrimenti

Questa codifica è un'applicazione diretta del **One Hot Encoding** per feature categoriali multivalore.

### Similarità del coseno

La **similarità tra vettori** è calcolata tramite la misura del coseno:

$$
\text{sim}(\vec{A}, \vec{B}) = \frac{\vec{A} \cdot \vec{B}}{\|\vec{A}\| \cdot \|\vec{B}\|}
$$

Dove:
- $\vec{A} \cdot \vec{B}$ è il prodotto scalare tra i due vettori
- $\|\vec{A}\|$ e $\|\vec{B}\|$ sono le loro norme euclidee

Questa metrica restituisce un valore tra 0 e 1 che indica quanto due vettori sono **orientati nella stessa direzione**. Non dipende dalla magnitudine, rendendola adatta per vettori binari.

### Prossimità semantica

Il concetto di **prossimità** viene dunque tradotto in termini geometrici: due film sono considerati simili se occupano posizioni vicine (in termini angolari) nello spazio generato dai generi.

![Similarità del coseno tra vettori](asset/cosine-similarity-vectors.original.jpg)

La funzione `raccomanda_simili` restituisce i top-N film più vicini al titolo di input secondo questa metrica.


In [6]:
import math

# Crea un vocabolario ordinato di tutti i generi presenti nel dataset (senza duplicati)
def costruisci_vocabolario_generi(dati):
    generi = set()
    for record in dati:
        for g in record['listed_in']:
            generi.add(g.strip().lower())
    return sorted(list(generi))

# Converte i generi di un film in un vettore binario in base al vocabolario
def vettore_generi(record, vocabolario):
    generi_film = set(g.lower() for g in record['listed_in'])
    return [1 if g in generi_film else 0 for g in vocabolario]

# Calcola la similarità coseno tra due vettori (quanto sono orientati nella stessa direzione)
def similarita_coseno(v1, v2):
    dot = sum(a*b for a, b in zip(v1, v2))
    norm1 = math.sqrt(sum(a*a for a in v1))
    norm2 = math.sqrt(sum(b*b for b in v2))
    if norm1 == 0 or norm2 == 0:
        return 0
    return dot / (norm1 * norm2)

# Cerca un film nel dataset confrontando il titolo in modo case-insensitive
def trova_record_per_titolo(dati, titolo_input):
    titolo_input = titolo_input.strip().lower()
    for record in dati:
        if record['title'].strip().lower() == titolo_input:
            return record
    return None

# Mostra i top-N titoli più simili a quello scelto, basandosi sui generi
def raccomanda_simili(dati, titolo_input, top_n=10):
    vocabolario = costruisci_vocabolario_generi(dati)
    film_richiesto = trova_record_per_titolo(dati, titolo_input)

    if film_richiesto is None:
        print("Titolo non trovato.")
        return

    vettore_input = vettore_generi(film_richiesto, vocabolario)
    risultati = []

    for record in dati:
        if record['title'] == film_richiesto['title']:
            continue
        v = vettore_generi(record, vocabolario)
        sim = similarita_coseno(vettore_input, v)
        risultati.append((record['title'], sim, record['listed_in']))

    top = sorted(risultati, key=lambda x: x[1], reverse=True)[:top_n]

    print(f"\nTitoli più simili a: {film_richiesto['title']} {film_richiesto['listed_in']}\n")
    for titolo, score, genres in top:
        print(f"{titolo} — similarità: {round(score, 2)} -> {genres}\n")

print("GENERI: ", costruisci_vocabolario_generi(dati), "\n")
print(dati[7659]['title'], '->', dati[7659]['listed_in'])
print("VETTORE:", vettore_generi(dati[7659], costruisci_vocabolario_generi(dati)), "\n")
print("Cos Similarity: ", similarita_coseno([0, 1, 0], [1, 0, 0]), "\n")

GENERI:  ['action & adventure', 'anime features', 'anime series', 'british tv shows', 'children & family movies', 'classic & cult tv', 'classic movies', 'comedies', 'crime tv shows', 'cult movies', 'documentaries', 'docuseries', 'dramas', 'faith & spirituality', 'horror movies', 'independent movies', 'international movies', 'international tv shows', "kids' tv", 'korean tv shows', 'lgbtq movies', 'movies', 'music & musicals', 'reality tv', 'romantic movies', 'romantic tv shows', 'sci-fi & fantasy', 'science & nature tv', 'spanish-language tv shows', 'sports movies', 'stand-up comedy', 'stand-up comedy & talk shows', 'teen tv shows', 'thrillers', 'tv action & adventure', 'tv comedies', 'tv dramas', 'tv horror', 'tv mysteries', 'tv sci-fi & fantasy', 'tv shows', 'tv thrillers'] 

Once Upon a Time in the West -> ['Action & Adventure', 'Classic Movies', 'International Movies']
VETTORE: [1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
titolo = input("Inserisci il titolo di un film o serie: ")
raccomanda_simili(dati, titolo)

## Espansione dello spazio vettoriale con feature categoriali multiple

Finora lo spazio delle feature era limitato ai generi. In questa fase, estendiamo la rappresentazione vettoriale di ogni titolo includendo altre feature categoriali discrete:

- `rating` (es. TV-MA, PG, etc.)
- `type` (Movie o TV Show)
- `director` (filtrando solo i **top-N più frequenti**)

### Approccio

- Le feature vengono one-hot encodate esattamente come fatto per i generi.
- I valori di `rating` e `type` sono direttamente binarizzati.
- Per `director`, si evita la codifica completa (troppo dispersiva) e si selezionano solo i più frequenti tramite [`Counter`](https://docs.python.org/3/library/collections.html#collections.Counter).

La funzione `vettore_completo` concatena tutti i sottovettori binari in un unico vettore ad alta dimensionalità che rappresenta ogni contenuto in modo più ricco.

### Similarità estesa

La funzione `raccomanda_simili_esteso` sfrutta questa rappresentazione multidimensionale per calcolare la **similarità del coseno** su uno spazio vettoriale più articolato. Il sistema è ora in grado di cogliere affinità non solo tematiche (genere), ma anche stilistiche (regia), strutturali (tipo) e di target (rating).

Questo rappresenta un primo passo concreto verso un sistema **ibrido basato su contenuto**, più sofisticato del semplice filtraggio su un attributo.

## Uso di Counter

```python
from collections import Counter

nomi = ['Luca', 'Anna', 'Luca', 'Marco', 'Anna', 'Luca']
conteggio = Counter(nomi)
print(conteggio)

# output
Counter({'Luca': 3, 'Anna': 2, 'Marco': 1})

```

Versione senza Counter

```python
director_counter = {}

for r in dati:
    if r['director']:
        nome = r['director'].strip()
        if nome not in director_counter:
            director_counter[nome] = 1
        else:
            director_counter[nome] += 1

```

In [ ]:
from collections import Counter

# Restituisce i vocabolari unici ordinati per rating, tipo, e i top N registi più frequenti
def costruisci_vocabolari_categorici(dati, top_n_directors=20):
    ratings = set()
    types = set()
    director_counter = Counter()

    for r in dati:
        # Questi if servono ad evitare di avere valori vuoti nei vari set
        if r['rating']:
            ratings.add(r['rating'].strip())
        if r['type']:
            types.add(r['type'].strip())
        if r['director']:
            nomi = [n.strip() for n in r['director'].split(',')]
            for nome in nomi:
                director_counter[nome] += 1

    top_directors = [d for d, _ in director_counter.most_common(top_n_directors)]

    return sorted(list(ratings)), sorted(list(types)), top_directors


def vettore_completo(record, voc_generi, voc_rating, voc_type, voc_directors):
    v_gen = vettore_generi(record, voc_generi)

    v_rat = [1 if record['rating'] == r else 0 for r in voc_rating]
    v_typ = [1 if record['type'] == t else 0 for t in voc_type]
    v_dir = [1 if d in record['director'] else 0 for d in voc_directors]

    return v_gen + v_rat + v_typ + v_dir

def raccomanda_simili_esteso(dati, titolo_input, top_n=5):
    voc_gen = costruisci_vocabolario_generi(dati)
    voc_rat, voc_typ, voc_dir = costruisci_vocabolari_categorici(dati)

    film_richiesto = trova_record_per_titolo(dati, titolo_input)
    if film_richiesto is None:
        print("Titolo non trovato.")
        return

    v_input = vettore_completo(film_richiesto, voc_gen, voc_rat, voc_typ, voc_dir)

    risultati = []
    for r in dati:
        if r['title'] == film_richiesto['title']:
            continue
        v_altro = vettore_completo(r, voc_gen, voc_rat, voc_typ, voc_dir)
        sim = similarita_coseno(v_input, v_altro)
        risultati.append((r['title'], sim))

    top = sorted(risultati, key=lambda x: x[1], reverse=True)[:top_n]

    print(f"\nTitoli più simili a: {film_richiesto['title']}\n")
    for titolo, score in top:
        print(f"{titolo} — similarità: {round(score, 2)}\n")

top_n_directors = 20

ratings, types, directors = costruisci_vocabolari_categorici(dati, top_n_directors)

print("RATING: ", ratings, "\n")
print("TYPES: ", types, "\n")
print(f"TOP {top_n_directors} DIRECTORS: ", directors, "\n")


once = dati[7659]

print(f"{once['title']}, girato da {once['director']}; generi: {once['listed_in']}" )
generi = costruisci_vocabolario_generi(dati)
vettore = vettore_completo(
    once, 
    generi,
    ratings,
    types,
    directors
)

print(vettore)

In [ ]:
titolo = input("Inserisci il titolo di un film o serie: ")
raccomanda_simili_esteso(dati, titolo)

## Raccomandazione tramite analisi testuale: descrizioni e TF-IDF

A differenza delle feature categoriali precedenti, il campo `description` è testuale e necessita di un trattamento diverso. Usiamo il modello **TF-IDF** (Term Frequency – Inverse Document Frequency) per rappresentare ogni descrizione come un vettore sparso che evidenzia i termini più rilevanti.

### 1. Tokenizzazione e normalizzazione

La funzione `tokenizza_testo` converte la descrizione in una lista di token, rimuovendo punteggiatura e portando tutto in minuscolo. Questo step è cruciale per costruire un vocabolario consistente.

### 2. Term Frequency (TF)

Per ogni film, viene calcolata la **frequenza relativa** di ciascuna parola rispetto al totale delle parole nella descrizione:

$$
\text{TF}(t, d) = \frac{f_{t,d}}{\sum_{t'} f_{t',d}}
$$

### 3. Inverse Document Frequency (IDF)

L’IDF penalizza i termini troppo comuni nell’intero corpus, dando maggior peso a parole distintive:

$$
\text{IDF}(t) = \log \left( \frac{N}{1 + \text{df}(t)} \right)
$$

dove:
- $N$ è il numero totale di documenti
- $\text{df}(t)$ è il numero di documenti in cui il termine $t$ appare

### 4. Vettori TF-IDF

Ogni descrizione viene rappresentata come un dizionario sparso:

$$
\text{TF-IDF}(t, d) = \text{TF}(t, d) \cdot \text{IDF}(t)
$$

Questo consente di rappresentare contenuti con lunghezze diverse in uno spazio comune, evidenziando le parole semanticamente rilevanti.

### 5. Similarità del coseno su vettori sparsi

La similarità viene calcolata solo sulle parole in comune tra due descrizioni, evitando la costruzione esplicita di vettori densi di dimensione pari al vocabolario completo:

$$
\cos(\theta) = \frac{\sum_{i} A_i B_i}{\|A\| \cdot \|B\|}
$$

### 6. Raccomandazione semantica

`raccomanda_simili_descrizione` confronta la descrizione di un titolo con tutte le altre e restituisce i titoli semanticamente più vicini, sfruttando esclusivamente la rappresentazione testuale.

Questo approccio consente di cogliere **affinità semantiche latenti** non esplicitamente codificate nelle feature strutturate.


## Esempio manuale su vocabolario ridotto

Per chiarire il funzionamento del modello TF-IDF, consideriamo un sottoinsieme semplificato di tre descrizioni fittizie:

1. "A cat sits on the mat"
2. "The dog lies on the mat"
3. "A cat and a dog play together"

Supponiamo che il vocabolario risultante dalla tokenizzazione sia:

```python
["cat", "dog", "mat", "on", "the", "sits", "lies", "play", "together"]
```

### 🔹 Calcolo TF

Per la frase 1:

Totale parole = 6  
Parole: ["a", "cat", "sits", "on", "the", "mat"]

Dopo rimozione di stopword come "a", rimangono:

- TF("cat") = 1/6
- TF("sits") = 1/6
- TF("on") = 1/6
- TF("the") = 1/6
- TF("mat") = 1/6

### 🔹 Calcolo IDF

Numero documenti $N = 3$

| Termine   | Document Frequency (DF) | IDF                        |
|-----------|--------------------------|----------------------------|
| "cat"     | 2                        | $\log(3 / (1 + 2)) = 0$    |
| "sits"    | 1                        | $\log(3 / 2) \approx 0.405$|
| "mat"     | 2                        | $\log(3 / 3) = 0$          |
| "on"      | 2                        | $\log(3 / 3) = 0$          |
| "the"     | 3                        | $\log(3 / 4) < 0$          |

### 🔹 Costruzione del vettore TF-IDF

Moltiplichiamo TF per IDF per ciascun termine. I termini assenti non sono inclusi:

| Termine   | TF    | IDF     | TF-IDF    |
|-----------|-------|---------|-----------|
| "cat"     | 1/6   | 0.000   | 0.000     |
| "sits"    | 1/6   | 0.405   | 0.0675    |
| "mat"     | 1/6   | 0.000   | 0.000     |
| "on"      | 1/6   | 0.000   | 0.000     |
| "the"     | 1/6   | < 0     | 0.000     |

### 🔹 Vettore finale

Nel vocabolario completo di 9 termini, la frase 1 sarà rappresentata così:

**Forma densa (posizionale):**

```python
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0675, 0.0, 0.0, 0.0]
```


**Forma sparsa (più efficiente):**

```python
{"sits": 0.0675}
```

### Interpretazione

Termini comuni e distribuiti su molti documenti, come "cat", "mat" e "the", ricevono un peso vicino a zero: il modello li riconosce come poco informativi. Al contrario, termini più rari e distintivi come "sits" ottengono un peso maggiore nel vettore TF-IDF.

Questa rappresentazione enfatizza la specificità lessicale di ciascun documento, consentendo al sistema di raccomandazione di identificare affinità semantiche non evidenti nelle feature strutturate.



In [ ]:
def tokenizza_testo(testo):
    testo = testo.lower()
    simboli = '.,!?()[]{}:;\"\''
    for s in simboli:
        testo = testo.replace(s, '')
    parole = testo.split()
    return parole

def calcola_tf(dataset):
    tf_per_film = []

    for record in dataset:
        parole = tokenizza_testo(record['description']) if record['description'] else []
        totale = len(parole)
        tf = {}
        for p in parole:
            tf[p] = tf.get(p, 0) + 1
        if totale > 0:
            for p in tf:
                tf[p] /= totale
        tf_per_film.append(tf)

    return tf_per_film

def calcola_idf(tf_per_film):
    df = {}
    N = len(tf_per_film)

    for tf in tf_per_film:
        for parola in tf.keys():
            df[parola] = df.get(parola, 0) + 1

    idf = {}
    for parola, conteggio in df.items():
        idf[parola] = math.log(N / (1 + conteggio))

    return idf

def calcola_tfidf(tf_per_film, idf):
    vettori = []

    for tf in tf_per_film:
        vettore = {}
        for parola in tf:
            vettore[parola] = tf[parola] * idf.get(parola, 0)
        vettori.append(vettore)

    return vettori

def similarita_coseno_sparsa(v1, v2):
    parole_comuni = set(v1.keys()) & set(v2.keys())
    numeratore = sum(v1[p] * v2[p] for p in parole_comuni)

    norm1 = math.sqrt(sum(v**2 for v in v1.values()))
    norm2 = math.sqrt(sum(v**2 for v in v2.values()))

    if norm1 == 0 or norm2 == 0:
        return 0.0

    return numeratore / (norm1 * norm2)


def raccomanda_simili_descrizione(dataset, vettori_tfidf, titolo_input, top_n=5):
    titolo_input = titolo_input.strip().lower()
    index = next((i for i, r in enumerate(dataset) if r['title'].strip().lower() == titolo_input), None)

    if index is None:
        print("Titolo non trovato.")
        return

    v_input = vettori_tfidf[index]
    risultati = []

    for i, v in enumerate(vettori_tfidf):
        if i == index:
            continue
        sim = similarita_coseno_sparsa(v_input, v)
        risultati.append((dataset[i]['title'], sim, dataset[i]['description']))

    top = sorted(risultati, key=lambda x: x[1], reverse=True)[:top_n]

    print(f"\nTitoli più simili a '{dataset[index]['title']}' per descrizione:\n")
    for titolo, score, description in top:
        print(f"{titolo} — similarità: {round(score, 2)}\n{description}\n\n")

In [ ]:
tf = calcola_tf(dati)
idf = calcola_idf(tf)
tfidf = calcola_tfidf(tf, idf)

print(dati[7659]['description'])
print(json.dumps(tf[7659], indent=4))
print(f"Term Frequency for 'his' -> {tf[7659]['his']}")

In [ ]:
print(idf['in'])

In [ ]:
tfidf

In [ ]:
titolo = input("Inserisci un titolo: ")
print(once['description'])
raccomanda_simili_descrizione(dati, tfidf, titolo)

## Raccomandazione ibrida: contenuto testuale + strutturale

Questa fase integra due rappresentazioni distinte di ciascun titolo:

- Un **vettore sparso TF-IDF** basato sulla descrizione testuale
- Un **vettore strutturato** binario costruito su generi, tipo, rating e regista

### Fusione dei segnali: similarità pesata

La funzione `similarita_pesata` calcola una **combinazione lineare** tra due misure di similarità del coseno:

$$
\text{sim}_{\text{ibrida}} = \alpha \cdot \text{sim}_{\text{TFIDF}} + (1 - \alpha) \cdot \text{sim}_{\text{struct}}
$$

dove:
- $\text{sim}_{\text{TFIDF}}$ è la similarità del coseno tra descrizioni testuali
- $\text{sim}_{\text{struct}}$ è la similarità del coseno tra vettori categoriali (generi, rating, ecc.)
- $\alpha$ è il **peso assegnato al contenuto testuale**, tipicamente $\alpha = 0.7$

### Algoritmo

La funzione `raccomanda_ibrido`:

1. Recupera sia la descrizione vettorializzata (`v_tfidf`) sia la rappresentazione strutturale (`v_struct`) del titolo richiesto
2. Confronta queste due rappresentazioni con tutti gli altri titoli del dataset
3. Calcola la similarità pesata
4. Restituisce i titoli con la similarità ibrida più alta

### Vantaggi del modello ibrido

- **Complementarietà semantica**: le descrizioni testuali catturano significati che non emergono da generi o classificazioni.
- **Robustezza strutturale**: i metadati strutturati compensano descrizioni brevi o poco informative.
- Il parametro $\alpha$ consente di **regolare dinamicamente** il bilanciamento tra le due fonti di informazione.

Questo approccio è una forma semplificata ma efficace di **content-based hybrid recommendation**.


In [ ]:
def similarita_pesata(v_tfidf1, v_struct1, v_tfidf2, v_struct2, peso_tfidf=0.7):
    sim_tfidf = similarita_coseno_sparsa(v_tfidf1, v_tfidf2)
    sim_struct = similarita_coseno(v_struct1, v_struct2)
    return peso_tfidf * sim_tfidf + (1 - peso_tfidf) * sim_struct


def raccomanda_ibrido(dati, vettori_tfidf, titolo_input, peso_tfidf=0.7, top_n=5):
    voc_gen = costruisci_vocabolario_generi(dati)
    voc_rat, voc_typ, voc_dir = costruisci_vocabolari_categorici(dati)

    titolo_input = titolo_input.strip().lower()
    index = next((i for i, r in enumerate(dati) if r['title'].strip().lower() == titolo_input), None)

    if index is None:
        print("Titolo non trovato.")
        return

    v_tfidf_input = vettori_tfidf[index]
    v_struct_input = vettore_completo(dati[index], voc_gen, voc_rat, voc_typ, voc_dir)

    risultati = []
    for i, r in enumerate(dati):
        if i == index:
            continue

        v_tfidf = vettori_tfidf[i]
        v_struct = vettore_completo(r, voc_gen, voc_rat, voc_typ, voc_dir)

        sim = similarita_pesata(v_tfidf_input, v_struct_input, v_tfidf, v_struct, peso_tfidf)
        risultati.append((r['title'], sim))

    top = sorted(risultati, key=lambda x: x[1], reverse=True)[:top_n]

    print(f"\nTitoli simili a '{dati[index]['title']}' (pesi: descrizione {peso_tfidf}, categorie {1 - peso_tfidf}):\n")
    for titolo, score in top:
        print(f"{titolo} — similarità: {round(score, 2)}")




In [ ]:
titolo = input("Inserisci il titolo di un film o serie: ")
raccomanda_ibrido(dati, tfidf, titolo, peso_tfidf=0.4)

## Raccomandazione semantica con Sentence Embeddings

In questa fase il sistema utilizza rappresentazioni dense delle descrizioni testuali, generate da modelli di tipo Transformer addestrati per catturare la semantica delle frasi. Ogni descrizione viene convertita in un vettore numerico ad alta dimensionalità che riflette il significato latente del testo, non solo le parole che lo compongono.

### Embedding semantici

Utilizzando un modello pre-addestrato come "all-distilroberta-v1", ogni descrizione viene proiettata in uno spazio vettoriale in cui frasi simili (anche se lessicalmente diverse) risultano vicine. Questi modelli sono stati ottimizzati per produrre embedding confrontabili tramite misure geometriche come la similarità del coseno.

A differenza del TF-IDF, che si basa su frequenze e presenze di termini, gli embeddings sono in grado di cogliere:

- sinonimie e parafrasi
- relazioni concettuali non esplicite
- contesti sintattici

Ad esempio, frasi come “Two kids are lost in space” e “A pair of children explore the galaxy” risultano semanticamente simili, pur non condividendo parole chiave.

### Raccomandazione su base semantica

Una volta ottenuti gli embedding delle descrizioni, si può confrontare qualsiasi titolo con tutti gli altri calcolando la similarità tra i relativi vettori. Questo approccio consente raccomandazioni basate sul significato del contenuto, indipendentemente dai generi o da altre etichette esplicite.

### Modello ibrido: embedding + metadati

La raccomandazione semantica può essere ulteriormente potenziata integrando anche la rappresentazione strutturale (generi, rating, tipo, regista). Si definisce così una similarità pesata che combina due contributi:

- similarità semantica tra descrizioni
- similarità strutturale basata su feature categoriali

La combinazione avviene tramite una media pesata tra le due componenti, regolabile tramite un parametro che definisce l'importanza relativa del contenuto testuale rispetto alla struttura.

Questo modello ibrido consente di sfruttare punti di forza complementari: la profondità semantica degli embedding e l’informazione esplicita contenuta nei metadati.


In [ ]:
!pip install sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer

# modello = SentenceTransformer('all-MiniLM-L6-v2')  # Veloce e buono
modello = SentenceTransformer('all-distilroberta-v1')

In [ ]:
def genera_embedding_descrizioni(dati, modello):
    descrizioni = [r['description'] if r['description'] else '' for r in dati]
    return modello.encode(descrizioni, show_progress_bar=True)

In [ ]:
from numpy import dot
from numpy.linalg import norm

def similarita_coseno_vec(v1, v2):
    if norm(v1) == 0 or norm(v2) == 0:
        return 0.0
    return dot(v1, v2) / (norm(v1) * norm(v2))


In [ ]:
def raccomanda_embedding(dati, embeddings, titolo_input, top_n=5):
    titolo_input = titolo_input.strip().lower()
    index = next((i for i, r in enumerate(dati) if r['title'].strip().lower() == titolo_input), None)

    if index is None:
        print("Titolo non trovato.")
        return

    v_input = embeddings[index]
    risultati = []

    for i, v in enumerate(embeddings):
        if i == index:
            continue
        sim = similarita_coseno_vec(v_input, v)
        risultati.append((dati[i]['title'], sim))

    top = sorted(risultati, key=lambda x: x[1], reverse=True)[:top_n]

    print(f"\nTitoli simili a '{dati[index]['title']}' (con embedding semantico):\n")
    for titolo, score in top:
        print(f"{titolo} — similarità: {round(score, 2)}")


In [ ]:
embedding_descrizioni = genera_embedding_descrizioni(dati, modello)


In [ ]:
embedding_descrizioni[0].shape

In [ ]:

titolo = input("Inserisci il titolo di un film o serie: ")
raccomanda_embedding(dati, embedding_descrizioni, titolo)

In [ ]:
def raccomanda_embedding_ibrido(dati, embedding_descrizioni, titolo_input, peso_emb=0.7, top_n=5):
    voc_gen = costruisci_vocabolario_generi(dati)
    voc_rat, voc_typ, voc_dir = costruisci_vocabolari_categorici(dati)

    titolo_input = titolo_input.strip().lower()
    index = next((i for i, r in enumerate(dati) if r['title'].strip().lower() == titolo_input), None)

    if index is None:
        print("Titolo non trovato.")
        return

    v_emb_input = embedding_descrizioni[index]
    v_struct_input = vettore_completo(dati[index], voc_gen, voc_rat, voc_typ, voc_dir)

    risultati = []
    for i, r in enumerate(dati):
        if i == index:
            continue

        v_emb = embedding_descrizioni[i]
        v_struct = vettore_completo(r, voc_gen, voc_rat, voc_typ, voc_dir)

        sim_emb = similarita_coseno_vec(v_emb_input, v_emb)
        sim_struct = similarita_coseno(v_struct_input, v_struct)

        sim_finale = peso_emb * sim_emb + (1 - peso_emb) * sim_struct
        risultati.append((r['title'], sim_finale))

    top = sorted(risultati, key=lambda x: x[1], reverse=True)[:top_n]

    print(f"\nTitoli simili a '{dati[index]['title']}' (embedding + colonne):\n")
    for titolo, score in top:
        print(f"{titolo} — similarità: {round(score, 2)}")



In [ ]:
titolo = input("Inserisci un titolo: ")
raccomanda_embedding_ibrido(dati, embedding_descrizioni, titolo, peso_emb=0.4)

## Ultimo step - Profilazione utente e raccomandazione personalizzata

Fino ad ora la raccomandazione era centrata su un titolo fornito in input. Ora passiamo a un'impostazione più realistica: generare raccomandazioni **a partire da una lista di titoli preferiti** da parte dell’utente.

### Costruzione del profilo utente

Un profilo utente è rappresentato da un vettore che sintetizza le preferenze implicite contenute nei film preferiti. Questo avviene tramite:

- **Media degli embedding testuali**: aggregazione semantica delle descrizioni dei titoli scelti
- **Media dei vettori strutturali**: aggregazione binaria di generi, rating, tipo e registi associati

Queste due componenti rappresentano rispettivamente il profilo **semantico** e **strutturale** dell’utente.

### Raccomandazione ibrida basata sul profilo

La funzione di raccomandazione confronta ciascun titolo del dataset con il profilo dell’utente:

- La similarità viene calcolata sia nel **feature space semantico** (embedding) che nel **feature space strutturale**
- Il punteggio finale è una media pesata delle due componenti, regolata da un parametro che bilancia l’influenza del contenuto testuale rispetto ai metadati

Questo approccio permette di generare suggerimenti **personalizzati e diversificati**, andando oltre la semplice similarità con un singolo film.

### Considerazioni

- Il profilo è **dinamico**: ogni variazione nella lista di preferiti cambia la rappresentazione e, di conseguenza, l'intero set raccomandato.
- L’uso della media vettoriale, seppur semplice, si dimostra efficace per catturare la “direzione generale” delle preferenze dell’utente nello spazio semantico.
- È possibile estendere questo modello introducendo pesi sui film (es. valutazioni esplicite) o tecniche più avanzate di aggregazione (es. attention).

Questo modulo conclude la transizione verso un **sistema di raccomandazione completo**, in grado di operare su input testuale libero e adattarsi al profilo dell’utente.


In [1]:
def profilo_utente(film_preferiti, dati, embedding_descrizioni):
    film_input = [t.strip().lower() for t in film_preferiti]
    
    indici = [
        i for i, r in enumerate(dati)
        if r['title'].strip().lower() in film_input
    ]

    if not indici:
        print("Nessun titolo trovato.")
        return None

    vettori = [embedding_descrizioni[i] for i in indici]
    vettore_medio = sum(vettori) / len(vettori)
    return vettore_medio

def profilo_utente_ibrido(film_preferiti, dati, embedding_descrizioni, voc_gen, voc_rat, voc_typ, voc_dir):
    film_input = [t.strip().lower() for t in film_preferiti]

    indici = [
        i for i, r in enumerate(dati)
        if r['title'].strip().lower() in film_input
    ]

    if not indici:
        print("Nessun titolo trovato.")
        return None, None

    emb_vettori = [embedding_descrizioni[i] for i in indici]
    struct_vettori = [vettore_completo(dati[i], voc_gen, voc_rat, voc_typ, voc_dir) for i in indici]

    emb_media = sum(emb_vettori) / len(emb_vettori)
    struct_media = [sum(x)/len(struct_vettori) for x in zip(*struct_vettori)]

    return emb_media, struct_media

def raccomanda_ibrido_utente(dati, embedding_descrizioni, film_preferiti, peso_emb=0.7, top_n=5):
    voc_gen = costruisci_vocabolario_generi(dati)
    voc_rat, voc_typ, voc_dir = costruisci_vocabolari_categorici(dati)

    v_emb_user, v_struct_user = profilo_utente_ibrido(film_preferiti, dati, embedding_descrizioni, voc_gen, voc_rat, voc_typ, voc_dir)
    if v_emb_user is None:
        return

    film_input = [t.strip().lower() for t in film_preferiti]
    risultati = []

    for i, record in enumerate(dati):
        titolo = record['title'].strip().lower()
        if titolo in film_input:
            continue

        v_emb = embedding_descrizioni[i]
        v_struct = vettore_completo(record, voc_gen, voc_rat, voc_typ, voc_dir)

        sim_emb = similarita_coseno_vec(v_emb_user, v_emb)
        sim_struct = similarita_coseno(v_struct_user, v_struct)
        sim_finale = peso_emb * sim_emb + (1 - peso_emb) * sim_struct

        risultati.append((record['title'], sim_finale))

    top = sorted(risultati, key=lambda x: x[1], reverse=True)[:top_n]

    print("\n🎯 Raccomandazioni personalizzate (embedding + struttura):\n")
    for titolo, score in top:
        print(f"{titolo} — similarità: {round(score, 2)}")



In [ ]:
film_preferiti = [
    "Once upon a time in the west",
    "Dances with Wolves",
    "The Outlaw Josey Wales"
]

raccomanda_ibrido_utente(dati, embedding_descrizioni, film_preferiti, peso_emb=0.4)